### Install mandatory packages

In [ ]:
!pip install albumentations ultralytics

Verify GPU usage

In [ ]:
!nvidia-smi

### Imports

In [ ]:
import os
from ultralytics import YOLO
import albumentations as A
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks

### Mounting Data storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
RootDir = '/content/drive/MyDrive/Virtuosity/' # Modify this path to point to the root directory of your project in Google Drive
os.listdir(RootDir)

### Augmentation method

In [2]:
def get_spectrogram_augmentations(intensity="medium"):
    """
    Augmentations optimisées pour spectrogrammes de trills.
    Compatible avec Albumentations >= 1.4.0

    Parameters:
    -----------
    intensity : str
        "light", "medium" ou "strong"
    """

    if intensity == "light":
        print('Training with light augmentations')
        return A.Compose([
            # Gamma (meilleur que Brightness pour spectro)
            A.RandomGamma(gamma_limit=(90, 110), p=0.3),

            # CLAHE (améliore contraste local)
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.2),

            # Bruit très léger
            A.GaussNoise(var_limit=(1.0, 8.0), p=0.2),  # ✅ Retiré 'mean'

            # Blur léger
            A.GaussianBlur(blur_limit=(3, 5), p=0.15),

            # SpecAugment très léger (petits trous)
            A.CoarseDropout(
                num_holes_range=(1, 2),      # ✅ Nouveau paramètre
                hole_height_range=(10, 30),  # ✅ Nouveau paramètre
                hole_width_range=(5, 10),    # ✅ Nouveau paramètre
                fill_value=0,
                p=0.25
            ),
        ])

    elif intensity == "medium":
        print('Training with medium augmentations')
        return A.Compose([
            # === 1️⃣ Dynamique / Contraste ===
            A.RandomGamma(
                gamma_limit=(85, 115),
                p=0.4
            ),

            A.CLAHE(
                clip_limit=2.0,
                tile_grid_size=(8, 8),
                p=0.3
            ),

            # === 2️⃣ Bruit ===
            A.GaussNoise(
                var_limit=(1.0, 20.0),  # ✅ Retiré 'mean'
                p=0.35
            ),

            # === 3️⃣ Perte de résolution ===
            A.GaussianBlur(
                blur_limit=(3, 5),
                p=0.25
            ),

            # === 4️⃣ SpecAugment hybride ===
            # Petits trous (masking temporel + fréquentiel)
            A.CoarseDropout(
                num_holes_range=(1, 3),       # ✅ 1 à 3 trous
                hole_height_range=(15, 40),   # ✅ Hauteur (fréquence)
                hole_width_range=(5, 15),     # ✅ Largeur (temps)
                fill_value=0,
                p=0.35
            ),

            # Bandes fréquentielles (toute la largeur)
            A.CoarseDropout(
                num_holes_range=(1, 1),       # ✅ 1 bande
                hole_height_range=(20, 50),   # ✅ Hauteur variable
                hole_width_range=(600, 640),  # ✅ Presque toute la largeur
                fill_value=0,
                p=0.2
            ),
        ])

    else:  # "strong"
        print('Training with strong augmentations')
        return A.Compose([
            # === 1️⃣ Dynamique / Contraste ===
            A.RandomGamma(
                gamma_limit=(80, 120),
                p=0.5
            ),

            A.CLAHE(
                clip_limit=3.0,
                tile_grid_size=(8, 8),
                p=0.4
            ),

            # === 2️⃣ Bruit ===
            A.GaussNoise(
                var_limit=(5.0, 30.0),  # ✅ Retiré 'mean'
                p=0.45
            ),

            # === 3️⃣ Perte de résolution ===
            A.OneOf([
                A.GaussianBlur(blur_limit=(3, 7), p=1.0),
                A.MedianBlur(blur_limit=5, p=1.0),
            ], p=0.3),

            # === 4️⃣ SpecAugment fort ===
            # Petits trous multiples
            A.CoarseDropout(
                num_holes_range=(2, 5),       # ✅ 2 à 5 trous
                hole_height_range=(15, 50),   # ✅ Hauteur
                hole_width_range=(5, 20),     # ✅ Largeur
                fill_value=0,
                p=0.4
            ),

            # Bandes fréquentielles
            A.CoarseDropout(
                num_holes_range=(1, 2),       # ✅ 1 à 2 bandes
                hole_height_range=(25, 60),   # ✅ Hauteur
                hole_width_range=(600, 640),  # ✅ Toute la largeur
                fill_value=0,
                p=0.3
            ),

            # Bandes temporelles
            A.CoarseDropout(
                num_holes_range=(1, 1),       # ✅ 1 bande verticale
                hole_height_range=(600, 640), # ✅ Toute la hauteur
                hole_width_range=(30, 80),    # ✅ Largeur
                fill_value=0,
                p=0.25
            ),
        ])

### Main call for training model

In [ ]:
# model = YOLO("yolov8n.pt")
model_type = "yolov8s"
model = YOLO(model_type + ".pt")

# model_stem = model.split('.')[0]

custom_transforms = get_spectrogram_augmentations(intensity="medium")

image_sizes = [640]

for size in image_sizes :
  print(f"Training on size {size} images with a {model_type} model")
  model.train(
      data=os.path.join(RootDir, "split/data.yaml"),
      imgsz=size,
      epochs=150,
      batch=4 if size == 1024 else 8,  # Augmentez à 16 si vous avez assez de RAM
      device="cuda",
      workers=2,
      cache=True,  # ✅ Activé pour 600 images
      amp=True,

      # === AUGMENTATIONS ULTRALYTICS (géométriques) ===
      # Comme le concurrent n'en propose pas, gardons-les légères
      translate=0.1,     # ±15% translation (léger)
      scale=0.2,         # ±15% zoom
      fliplr=0.0,         # Pas de flip horizontal
      flipud=0.0,         # Pas de flip vertical
      mosaic=0.0,         # Désactivé
      hsv_h=0.0,
      hsv_s=0.0,
      hsv_v=0.0,          # Gamma gère déjà la luminosité
      degrees=0.0,        # Pas de rotation
      perspective=0.0,    # Pas de perspective
      erasing=0.0,        # CoarseDropout gère déjà le masking

      # === AUGMENTATIONS ALBUMENTATIONS ===
      augmentations=custom_transforms,

      # === AUTRES PARAMÈTRES ===
      project=os.path.join(RootDir, "runs_yolo"),
      name=f"trill_{model_type}_medium_aug_{size}_test",
      patience=30,
      # save=True,
      # save_period=10,

      # Optimizer settings (optionnel)
      optimizer='AdamW',
      lr0=0.001,
      lrf=0.01,
  )
